In [1]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import numpy as np
    import tensorflow as tf
    
    from elasticsearch import helpers
    
    from gensim.models import Word2Vec, KeyedVectors
    
    import nltk
    import re
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    
    import spacy
    import pickle

    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

Loaded  .. . . . . . . .


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/srinivas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open("/home/srinivas/Desktop/dataframe/the_data.pkl",'rb') as f:
     df= pickle.load(f)


In [3]:
df

,atom,country,product,query,fasttext_answer,fasttext_question,w2v_question,w2v_answer,bert_answer,bert_question,spacy_question,spacy_answer,use_question,use_answer
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.11014999728649855, 0.007775001227855682, 0...","[-0.06475830078125, 0.20947265625, -0.01521396...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.2899235, -0.121713676, 0.24831414, 0.076035...","[-0.24090976, 0.256362, 0.020495497, -0.099701...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.027756759896874428, 0.007806204259395599, -...","[0.04650423303246498, 0.03865881636738777, -0...."
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.10943333432078362, 0.006266666886707147, 0...","[-0.09993489583333333, 0.004964192708333333, 0...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[-0.04205227, -0.019552507, 0.16107942, 0.3579...","[-0.012048334, 0.32869837, -0.0176391, -0.0020...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.0189292561262846, -0.021205231547355652, -0...","[0.04650423303246498, 0.03865881636738777, -0...."
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.1116000022739172, -0.0011249997187405825, ...","[-0.037353515625, 0.00390625, 0.048828125, 0.1...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.37489668, 0.033670336, -0.09372824, 0.08862...","[0.028587498, 0.0662625, 0.1000235, 0.00710500...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.02716185711324215, 0.0028547781985253096, -...","[0.04650423303246498, 0.03865881636738777, -0...."
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...,"[-0.051349122751210084, 0.0010666667520509738,...","[0.000200001522898674, 0.0321500007994473, 0.1...","[-0.06671142578125, 0.0811767578125, 0.0469970...","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[0.050525643, 0.058479086, 0.17381626, 0.10562...","[-0.0468025, 0.19192249, -0.19600251, -0.04838...","[-0.072944276, 0.2486027, -0.19252801, 0.05586...","[0.04280677065253258, 0.031687188893556595, -0...","[0.040527839213609695, 0.058647628873586655, -..."
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...,"[-0.051349122751210084, 0.0010666667520509738,...","[-0.02942222191227807, 0.038144443132397204, 0...","[-0.03811306423611111, 0.14531114366319445, 0....","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[-0.14897719, 0.051382728, -0.06139521, 0.1016...","[-0.13795668, 0.115536675, -0.18193598, 0.0613...","[-0.072944276, 0.2486027, -0.19252801, 0.05586...","[0.003646381665021181, 0.06274382770061493, -0...","[0.040527839213609695, 0.058647628873586655, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2250,Description:\tLisinopril is a white to off-whi...,Canada,Zestril (lisinopril),Lisinospril is what kind of powder?,"[-0.0022642851940223147, -0.0563214278081432, ...","[-0.0706000030040741, -0.026100000366568565, 0...","[0.05889892578125, -0.0111083984375, 0.1536865...","[0.026881626674107144, 0.005759102957589286, 0...","[-0.02737657, -0.07943635, -0.22290449, 0.0618...","[-0.044409927, -0.16870317, -0.15328597, 0.170...","[-0.07918367, 0.24674332, 0.029136, -0.0801326...","[-0.28126448, 0.22462204, -0.064237304, -0.127...","[-0.0

In [4]:
country=df["country"]
country_set=set(country)
product=df["product"]
product_set=set(product)
print("countries : "+str(country_set))
print("products : "+str(product_set))

countries : {'United States', 'Canada', 'European Union'}
products : {'Zestril (lisinopril)', 'Plendil(Felodipine)', 'Rosuvastatin(Crestor)'}


In [5]:
fast_answer=df["fasttext_answer"]

In [6]:
len(fast_answer)

2255

In [7]:
w2v_answer=df["w2v_answer"]

In [8]:
len(w2v_answer)

2255

In [9]:
spacy_answer=df["spacy_answer"]

In [10]:
len(spacy_answer)

2255

In [11]:
bert_answer=df["bert_answer"]

In [12]:
len(bert_answer)

2255

In [13]:
use_answer=df["use_answer"]

In [14]:
len(use_answer)

2255

In [23]:
answer=df["atom"]

In [24]:
len(answer)

2255

In [25]:
country_rec=df["country"]

In [28]:
len(country_rec)
for i in range(0,len(country_rec)):
    country_rec[i]=country_rec[i].lower()


In [29]:
country_rec[0]

'canada'

In [30]:
product_rec=df["product"]

In [31]:
len(product_rec)

2255

In [32]:
for i in range(0,len(product_rec)):
    product_rec[i]=product_rec[i].lower()
product_rec[0]

'plendil(felodipine)'

In [15]:
print(tf.__version__)

1.15.0-rc1


In [16]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "fasttext_vector":{
         "type":"dense_vector",
         "dims":300
      },
        "w2v_vector":{
         "type":"dense_vector",
         "dims":300
      },
        "spacy_vector":{
         "type":"dense_vector",
         "dims":300
      },
        "bert_vector":{
         "type":"dense_vector",
         "dims":768
      },
        "use_vector":{
         "type":"dense_vector",
         "dims":512
      },
          "country":{  
            "type":"text"
         },   
          "product":{  
            "type":"text"
         }
    }
   }
}

In [17]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

python client created


In [18]:
#creating the database
IndexName = 'the_final_ensemble'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [19]:
my

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [the_final_ensemble/gOJq7_GcTAGGo4c8JEVZoQ] already exists',
    'index_uuid': 'gOJq7_GcTAGGo4c8JEVZoQ',
    'index': 'the_final_ensemble'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [the_final_ensemble/gOJq7_GcTAGGo4c8JEVZoQ] already exists',
  'index_uuid': 'gOJq7_GcTAGGo4c8JEVZoQ',
  'index': 'the_final_ensemble'},
 'status': 400}

In [20]:
df.columns

Index(['atom', 'country', 'product', 'query', 'fasttext_answer',
       'fasttext_question', 'w2v_question', 'w2v_answer', 'bert_answer',
       'bert_question', 'spacy_question', 'spacy_answer', 'use_question',
       'use_answer'],
      dtype='object')

In [ ]:
df22 = df.to_dict('records')
df22

In [ ]:
len(df22)

In [33]:
arr=[]
for i in range(0,len(answer)):
     arr.append({
    'answer':answer[i],
    'fasttext_vector':fast_answer[i],
    'w2v_vector':w2v_answer[i],
    'spacy_vector':spacy_answer[i],
    'bert_vector':bert_answer[i],
    'use_vector':use_answer[i],
    'country':country_rec[i].lower().strip(),
    'product':product_rec[i].lower().strip()})

In [35]:
len(arr)

2255

In [36]:
for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [37]:
resp

{'_index': 'the_final_ensemble',
 '_type': '_doc',
 '_id': '2254',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 2254,
 '_primary_term': 1}

In [38]:
que="what is Crestor used for in canada?"
print(que)
temp_prod=df["product"]
temp_country=df["country"]
product_list=list(set(temp_prod))
print(product_list)
country_list=list(set(temp_country))
print(country_list)

what is Crestor used for in canada?
['zestril (lisinopril)', 'rosuvastatin(crestor)', 'plendil(felodipine)']
['canada', 'united states', 'european union']


In [39]:
def list_generator():    
    temp_prod=df["product"]
    temp_country=df["country"]

    product_list=list(set(temp_prod))
    country_list=list(set(temp_country))

    for i in range(0,len(product_list)):
        product_list[i]=product_list[i].lower()

    for i in range(0,len(country_list)):
        country_list[i]=country_list[i].lower().strip()

    prods=[]

    for x in product_list:
        temp=x.split("(")
        for i in range(0,len(temp)):
            temp[i]=temp[i].replace(")","")
            temp[i]=temp[i].strip()
            prods.append(temp[i])
    return [prods,country_list]

    

In [40]:
list_generator()

[['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine'],
 ['canada', 'united states', 'european union']]

In [49]:
global spacy_question
spacy_question=df["spacy_question"]
print(len(spacy_question))
print(len(spacy_question[0]))

2255
300


In [50]:
global bert_question
bert_question=df["bert_question"]
print(len(bert_question))
print(len(bert_question[0]))

2255
768


In [51]:
global fasttext_question
fasttext_question=df["fasttext_question"]
print(len(fasttext_question))
print(len(fasttext_question[0]))

2255
300


In [52]:
global w2v_question
w2v_question=df["w2v_question"]
print(len(w2v_question))
print(len(w2v_question[0]))

2255
300


In [53]:
global use_question
use_question=df["use_question"]
print(len(use_question))
print(len(use_question[0]))

2255
512


In [54]:
def query_creator(que,lists,index): 
    global spacy_question
    global use_question
    global w2v_question
    global fasttext_question
    global bert_question
    prods=lists[0]
    country_list=lists[1]
    print(prods)
    print(country_list)
    
    spacy_vec=spacy_question[index]
    w2v_vec=w2v_question[index]
    fasttext_vec=fasttext_question[index]
    bert_vec=bert_question[index]
    use_vec=use_question[index]
    #print(que_vec)
    #print(que)
    que=que.lower()
    p=""
    c=""
    for x in prods:
        if x in que:
            p=x
            print(p)
            break
    for x in country_list:
        if x in que:
            c=x
            print(c)

    script_query={}
    if p!="" and c!="":
        print("1...........")
        script_query = {
        "script_score": {
        "query": {
    "bool": {
      "must": [
        {
          "match": {
            "product": p
          }
        }
      ],
      "should": [
        {
          "match": {
            "country": c
          }
        }
      ]
    }
  },
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector'])))+(Math.abs(cosineSimilarity(params.bert_vector, doc['bert_vector'])))+(Math.abs(cosineSimilarity(params.use_vector, doc['use_vector']))))/5",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec,"bert_vector": bert_vec,"use_vector": use_vec}
        }}}

    elif p!="" and c=="":
        print("2...........")
        script_query = {
        "script_score": {
        "query": {"match": {
              "product": {
                "query":p
              }}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector'])))+(Math.abs(cosineSimilarity(params.bert_vector, doc['bert_vector'])))+(Math.abs(cosineSimilarity(params.use_vector, doc['use_vector']))))/5",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec,"bert_vector": bert_vec,"use_vector": use_vec}
        }}}
    elif p=="" and c!="":
        print("3...........")
        script_query = {
        "script_score": {
        "query": {"match": {
              "country": {
                "query":c
              }}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector'])))+(Math.abs(cosineSimilarity(params.bert_vector, doc['bert_vector'])))+(Math.abs(cosineSimilarity(params.use_vector, doc['use_vector']))))/5",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec,"bert_vector": bert_vec,"use_vector": use_vec}
        }}}
    else:
        print("4...........")
        script_query = {
        "script_score": {
        "query": {"match_all": {}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector'])))+(Math.abs(cosineSimilarity(params.bert_vector, doc['bert_vector'])))+(Math.abs(cosineSimilarity(params.use_vector, doc['use_vector']))))/5",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec,"bert_vector": bert_vec,"use_vector": use_vec}
        }}}
    return  script_query

In [55]:
lists=list_generator()

In [56]:
lists

[['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine'],
 ['canada', 'united states', 'european union']]

In [59]:
que=df["query"][0]
que

'What is a calcium ion influx inhibitor?'

In [60]:
script_query=query_creator(que,lists,0)
script_query

['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........


{'script_score': {'query': {'match_all': {}},
  'script': {'source': "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector'])))+(Math.abs(cosineSimilarity(params.bert_vector, doc['bert_vector'])))+(Math.abs(cosineSimilarity(params.use_vector, doc['use_vector']))))/5",
   'params': {'spacy_vector': array([-2.40909755e-01,  2.56361991e-01,  2.04954967e-02, -9.97011960e-02,
           -5.69734097e-01,  5.51467478e-01, -1.77477598e-01,  1.56602502e-01,
            2.15511993e-02,  4.62359995e-01,  5.67369908e-02, -1.25637501e-01,
            3.02442491e-01,  5.31389773e-01,  6.63620710e-01,  1.53209999e-01,
            1.82684988e-01,  1.98617244e+00, -1.39418006e-01,  7.12924451e-03,
            2.60667503e-01, -2.74022490e-01, -4.96464998e-01,  1.57325998e-01,
           -1.30727500e-01,  9.42209885e-02, -1.56278983e-01, -2.46014997

In [61]:
ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer","country","product"]}})

/home/srinivas/anaconda3/envs/success/lib/python3.6/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [62]:
ans

{'took': 776,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2255, 'relation': 'eq'},
  'max_score': 0.87329066,
  'hits': [{'_index': 'the_final_ensemble',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.87329066,
    '_source': {'country': 'canada',
     'product': 'plendil(felodipine)',
     'answer': 'PLENDIL (felodipine) is a calcium ion influx inhibitor (calcium channel blocker). Felodipine is a member of the dihydropyridine class of calcium channel blockers.'}},
   {'_index': 'the_final_ensemble',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.87329066,
    '_source': {'country': 'canada',
     'product': 'plendil(felodipine)',
     'answer': 'PLENDIL (felodipine) is a calcium ion influx inhibitor (calcium channel blocker). Felodipine is a member of the dihydropyridine class of calcium channel blockers.'}},
   {'_index': 'the_final_ensemble',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.8

In [63]:
def questions():
    count=0
    que_arr=[]
    for x in df["query"]:
        sent=x.split("\n")
        for q in sent:
            if re.search('[a-zA-Z]',q)!=None:
                que_arr.append([q,count])
        count+=1
    return que_arr

In [64]:
que_arr=questions()

In [65]:
len(que_arr)

2255

In [66]:
que_arr

[['What is a calcium ion influx inhibitor?', 0],
 ['What class of calcium channel blockers is Felodipine a member of?', 1],
 ['Felodipine is a member of which class of inhibitors?', 2],
 ['What is the therapeutic effect of Felodipine related to?', 3],
 ['How are the contractile processes of these tissues dependent upon the movement of extracellular calcium into the cells?',
  4],
 ['How does Felodipine blocks transmembrane influx of calcium ?', 5],
 ['Through what channel Felodipine blocks transmembrane influx of calcium ?',
  6],
 ['What process results in a reduction of free calcium ions available within cells of the tissues like cardiac muscle and vascular smooth muscle',
  7],
 ['What does Felodipine do not alter?', 8],
 ['In vitro studies show that the effects of what on contractile mechanisms are selective?',
  9],
 ['What is the effect on vascular smooth muscle?', 10],
 ['Negative inotropic effects can be detected in what animal?', 11],
 ['How Negative inotropic effects of felod

In [67]:

def rank(ans):
    required=ans['hits']
    main_arr=required["hits"]
    count=1
    ids__=dict()
    for x in main_arr:
        id_no= x["_id"]

        score=x["_score"]

        answer_=x["_source"]["answer"]
        
        ids__[id_no.strip()]=1/count
        count+=1
    return ids__
    

In [68]:
ranker=rank(ans)

In [69]:
ranker

{'0': 1.0,
 '1': 0.5,
 '2': 0.3333333333333333,
 '3': 0.25,
 '4': 0.2,
 '5': 0.16666666666666666,
 '6': 0.14285714285714285,
 '7': 0.125,
 '1094': 0.1111111111111111,
 '1095': 0.1}

In [ ]:
ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

In [70]:
def mrr():
    total=0
    count=0
    recall_count=0
    lists_all=list_generator()
    print(lists_all)
    for i in range(0,len(que_arr)):
        question_=que_arr[i][0].strip()
        print(question_)
        id_number=str(que_arr[i][1])

        script_query_curr=query_creator(question_,lists_all,i)
        ans=es.search(index=IndexName,body={"size": 10,"query": script_query_curr,"_source": {"includes": ["answer","product","country"]}})
        ranker=rank(ans)
        if id_number in ranker:
            total+=ranker[id_number]
            print(ranker[id_number])
            recall_count+=1
        else:
            print(0)
        count+=1

    print("mrr is"+str(total/count))
    print("recall is"+str(recall_count/count))
    return
        

In [71]:
mrr()

[['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine'], ['canada', 'united states', 'european union']]
What is a calcium ion influx inhibitor?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
1.0
What class of calcium channel blockers is Felodipine a member of?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
felodipine
2...........
0.5
Felodipine is a member of which class of inhibitors?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
felodipine
2...........
0
What is the therapeutic effect of Felodipine related to?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
felodipine
2...........
0
How are the contractile processes of these tissues dependent upo

0.2
How many times did the administration of a single dose of plain felidipine tablets increase as compared to administration with water?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
1.0
What does the Studies in healthy male volunteers showed during PLENDIL administration
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
plendil
2...........
0
What happened following the administration of a single dose of plain felodipine 5 mg tablets with 200 mL grapefruit juice
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
felodipine
2...........
0.3333333333333333
What is the average clearance of felodipine in elderly hypertensives?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
felod

0.125
What does the Studies in pregnant rabbits administered doses of 0.46, 1.2, 2.3 and 4.6 mg/kg/day of felodipine showed?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
felodipine
2...........
1.0
What was observed in a teratology study in Cynomolgus monkeys?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
1.0
What percentage of the fetuses had an abnormal position of the distal phalanges in about 40 percent?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0.5
What are the results of a teratology study in Cynomolgus monkeys
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0.1
In which patuent group prolongation of parturition with difficult labou

0.14285714285714285
What can induce gingival enlargement in patients with pronounced gingivitis and periodontitis?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
What can be avoided or reversed by measures of good oral hygiene and mechanical debridement of the teeth?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0.5
In which patients PLENDIL can induce gingival enlargement
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
plendil
2...........
0
By what measure gingival enlargement can be avoided or reversed
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
Can PLENDIL cause gingivitis.
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil

0.5
What can cause excessive peripheral vasodilation with marked hypotension and possibly bradycardia?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
1.0
What may cause excessive peripheral vasodilation with marked hypotension and possibly bradycardia in PLENDIL treatment
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
plendil
2...........
0.5
What do you need to do to manage a suspected drug overdose?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
1.0
How to manage suspected PLENDIL overdose
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
plendil
2...........
0.5
What should be swallowed whole and not crushed or chewed?
['zestril', 'lisinopril', 'rosuvastatin

...

1.0
In which patients glycemic control should be closely monitored during the first month of treatment with ZESTRIL ?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
zestril
2...........
0.5
What has been reported in patients dialysed with high-flux membranes?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0.3333333333333333
In which patients Anaphylactoid reactions have been reported
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
When Dialysis should be stopped immediately
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
What has been reported of patients experiencing life threatening anaphylactoid reactions during desensitizing treatment with

0.25
When further falls in blood pressure may also occur during zestril treatment?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
zestril
2...........
1.0
What is contraindicated in patients with diabetes and/or renal impairment?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
Wha
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
Dual Blockade of the Renin-Angiotensin-System (RAS) with ARBs or ACE inhibitors and aliskiren-containing drugs are contraindicated in which patients?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0.25
Why Co-administration of ARBs, ACE inhibitors or aliskiren- containing drugs is generally not recommended in other pati

0.25
What percentage of angioedema has been reported in patients receiving ZESTRIL?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
zestril
2...........
1.0
As a rare case what has been occurred in patients receiving zestril?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
zestril
2...........
0.3333333333333333
In hypertensive patients
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
1.0
what occurred in 0.8% of patients?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0.125
What was a cause for discontinuation of therapy in 0.3% of patients with what type of disease?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'unite

0
What is the usual starting dose in patients with essential hypertension?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
How many mg per day is a normal starting dose?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
What can be evaluated by measuring blood pressure prior to dosing to determine if satisfactory control is being maintained for 24 hours?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
What is the usual dosage range of zestril
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
zestril
2...........
0.1111111111111111
Can antihypertensive effect of zestril may diminish toward the end of the dosing interval ?
['zestril', 'lisinopril', 

0
What is ZESTRIL to be used in conjunction with?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
zestril
2...........
0
What must be initiated under close medical supervision?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
0
Why Blood pressure and renal function should be monitored, both before and during treatment with ZESTRIL ?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
zestril
2...........
0.3333333333333333
What should be reduced before beginning treatment?
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada', 'united states', 'european union']
4...........
1.0
Of what the Initiation of therapy requires consideration
['zestril', 'lisinopril', 'rosuvastatin', 'crestor', 'plendil', 'felodipine']
['canada',